bag of words


In [5]:
from collections import defaultdict
import numpy as np
import nltk
import re
from nltk.corpus import stopwords
nltk.download('stopwords')
print(stopwords.words('english'))
from nltk.stem.porter import *
from sklearn import linear_model
from sklearn.manifold import TSNE
import gzip
import json
import string
import math

import random
from sklearn.model_selection import train_test_split

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\boyiq\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [6]:
# from google.colab import drive
# drive.mount('/content/drive/')
# file_path = '/content/drive/My Drive/Digital_Music_5.json.gz'
file_path = 'Digital_Music_5.json.gz'

In [7]:
data = []
with gzip.open(file_path, 'rb') as file:
    for byte_line in file:
        line = byte_line.decode('utf-8').strip()
        review = json.loads(line)
        data.append(review)

In [8]:
data[0]

{'overall': 5.0,
 'vote': '3',
 'verified': True,
 'reviewTime': '06 3, 2013',
 'reviewerID': 'A2TYZ821XXK2YZ',
 'asin': '3426958910',
 'style': {'Format:': ' Audio CD'},
 'reviewerName': 'Garrett',
 'reviewText': 'This is awesome to listen to, A must-have for all Slayer fans..sadly needed to be a triple disc set..They have so many hits!!',
 'summary': 'Slayer Rules!',
 'unixReviewTime': 1370217600}

In [9]:
import pandas as pd
dataset = []
for d in data:
    if 'reviewText' not in d:
        continue
    if 'overall' not in d:
        d['overall'] = 5.0
    dataset.append({'reviewText':d['reviewText'], 'overall': d['overall']})
df = pd.DataFrame(dataset)

In [10]:
# preprocess the dataset's reviewtext by removing punctuation, stopwords


In [11]:
import nltk
from nltk.stem import SnowballStemmer
from nltk.corpus import stopwords
nltk.download('stopwords')

stop_words = stopwords.words('english')
stemmer = SnowballStemmer('english')
text_cleaning_re = "@\S+|https?:\S+|http?:\S|[^A-Za-z0-9]+"
import re

def preprocess(text, stem=False):
    text = re.sub(text_cleaning_re, ' ', str(text).lower()).strip()
    tokens = []
    for token in text.split():
        if token not in stop_words:
            if stem:
                tokens.append(stemmer.stem(token))
            else:
                tokens.append(token)
    return " ".join(tokens)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\boyiq\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [12]:
df.reviewText = df.reviewText.apply(lambda x : preprocess(x))

In [13]:
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction import text

# Bag of Words with bigrams and limited features
text_transformer = CountVectorizer(ngram_range=(1, 2),
                                   max_features=1000)

# TF-IDF Transformation
tfidf_transformer = TfidfTransformer()

# Pipeline: CountVectorizer -> TfidfTransformer
pipeline = Pipeline([
    ('vect', text_transformer),
    ('tfidf', tfidf_transformer)
])


In [14]:
# apply the BOW vectorization and TF-IDF transformation
X_text = pipeline.fit_transform(df.reviewText)

In [15]:
# 9:1 train-test split
X_train, X_test, y_train, y_test = train_test_split(X_text, df.overall, test_size=0.1, random_state=42)

In [16]:
X_train

<152660x1000 sparse matrix of type '<class 'numpy.float64'>'
	with 1530048 stored elements in Compressed Sparse Row format>

In [17]:
y_train

134858    5.0
104072    4.0
115766    5.0
66080     5.0
19492     4.0
         ... 
119879    1.0
103694    5.0
131932    5.0
146867    5.0
121958    5.0
Name: overall, Length: 152660, dtype: float64

# K-Nearest Neighbors with Cosine Similarity

In [18]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics.pairwise import cosine_similarity
# Function to predict using cosine similarity with KNN
def predict_with_cosine_similarity(X_train, y_train, X_test, k=10):
    # Calculate cosine similarity matrix
    similarity_matrix = cosine_similarity(X_test, X_train)

    # Predict ratings
    predictions = []
    for similarity in similarity_matrix:
        # Get top k indices of most similar items
        top_k_indices = np.argsort(similarity)[-k:]
        # Compute the mean of the k nearest neighbors
        predictions.append(np.mean(y_train.iloc[top_k_indices]))

    return predictions


In [20]:

# Predict ratings for test data
y_pred_cosine = predict_with_cosine_similarity(X_train, y_train, X_test, k=10)

In [22]:
# write the predicted ratings to a file
with open('cosine_predictions.txt', 'w') as f:
    for p in y_pred_cosine:
        f.write(str(p) + '\n')
        

In [24]:

# calculate mse score pairwise
from sklearn.metrics import mean_squared_error
mse = mean_squared_error(y_test, y_pred_cosine)
print(mse)

0.39787773389141073


# SVM on tfidf

In [ ]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

# Create and train the SVM model
svm_model = SVC(kernel='linear') # try other kenrnels
svm_model.fit(X_train, y_train)

# Predict ratings for test data
y_pred_svm = svm_model.predict(X_test)

# Evaluate the model
accuracy_svm = accuracy_score(y_test, y_pred_svm)
print(f"SVM Model Accuracy: {accuracy_svm}");

# Multinomial Naive Bayes Classifier on TF-IDF

In [26]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score

# Create and train the Naive Bayes model
nb_model = MultinomialNB()
nb_model.fit(X_train, y_train) # default alpha=1.0, 

# Predict ratings for test data
y_pred_nb = nb_model.predict(X_test)

# Evaluate the model (optional)
mse_nb = mean_squared_error(y_test, y_pred_nb)
print(f"Naive Bayes mse: {mse_nb}")



Naive Bayes mse: 0.5565642869775393
